### Criar o navegador 

In [65]:

#Imports necessários
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import pandas as pd

#Abrindo navegador
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)


In [66]:
caminho = os.getcwd()
arquivo = caminho + r"\files\consulta_juridica\index.html"



### Importar a base de dados

In [67]:
df = pd.read_excel(r'files/consulta_juridica/Processos.xlsx')

In [68]:
for linha in df.index:

    #Abrir a página index
    navegador.get(arquivo)


    # Simular ação hover (mouse em cima de um botão)

    botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
    ActionChains(navegador).move_to_element(botao).perform()

    cidade = df.loc[linha, "Cidade"]
    

    #Abrindo as cidades de maneira dinâmica
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

    #Detectando as abas
    aba_original = navegador.window_handles[0]

    indice = 1 + linha
    nova_aba = navegador.window_handles[indice]

    #Trocando de aba
    navegador.switch_to.window(nova_aba)

    #Preencher o formulário com os dados de busca
    navegador.find_element(By.ID, 'nome').send_keys(df.loc[linha, "Nome"])
    navegador.find_element(By.ID, 'advogado').send_keys(df.loc[linha, "Advogado"])
    navegador.find_element(By.ID, 'numero').send_keys(df.loc[linha, "Processo"])

    #Clicar em pesquisar 
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()

    #Confirmar a pesquisa (alert)
    alerta = navegador.switch_to.alert
    alerta.accept()

    #Aguardar o resultado da pesquisa
    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)


    texto_alerta = alerta.text

    if "Processo encontrado com sucesso" in texto_alerta:
        alerta.accept()
        df.loc[linha, "Status"] = "Encontrado"
    else:
        df.loc[linha, "Status"] = "Não Encontrado"
        alerta.accept()
    
    
    

In [69]:
navegador.quit()
df.head()
df.to_excel(r"files\consulta_juridica\ProcessosAtualizados.xlsx", index=False)